In [1]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://lumin-azure-openai-rnd.openai.azure.com/"
openai.api_version = "2024-02-01"
openai.api_key = "38002279fa574fbcba0fbc424ae056d5"

import os
from openai import AzureOpenAI
client = AzureOpenAI(
  azure_endpoint =  "https://lumin-azure-openai-rnd.openai.azure.com/",
  api_key="38002279fa574fbcba0fbc424ae056d5",  
  api_version="2024-02-01"
)

In [2]:
narrative_generation_rules = """Act as a narrative generator who always strictly follow the specified rules. Generate a concise, professional narrative describing the data in the given JSON input by keeping all important points for this business report for an analyst.
        Rule 1: Keep the narrative brief consisting of 1 to 4 sentences, using simple and professional language without repetition, and not exceeding 50 words, except for "why" questions. 
        Rule 2: If a date-type dimension is present, treat it as a trend line; otherwise, consider it as a distributional chart or bar chart.
        Rule 3: For trend line charts, the narrative must include the total measure value, the latest measure value, percentage change in the last date interval, and the highest single period climb in percentage.
        Rule 4: For distributional or bar charts, the narrative must provide the total measure value, dimension with the highest measure percentage, and the average measure value if present.
        Rule 5: If a filter is present in the input JSON, focus on it in the initial sentence of the narrative.
        Rule 6: When specifying filters, prioritize in this order in the narrative: location filter, date filter, and default filter.
        Rule 7: For charts with non-additive measures, concentrate on the top, second, and bottom players in the narrative.
        Rule 8: For ratio questions, the narrative must emphasize the ratio values.
        Rule 9: In share-type questions with bar charts, mention the top and bottom players in the narrative.
        Rule 10: For share-type questions with trend line charts, the narrative must include percentage share, interval, change in share, and share trend in the latest date interval, excluding other information.
        Rule 11: For growth-rate questions, focus on the latest measure value, final percentage change, and growth-rate trends in the narrative.
        Rule 12: For "why" questions, the narrative must detail on target change, changes in focus areas, and their contribution percentages. These narratives should be more elaborate.
        Rule 13: For kda question types, the narrative must include only the top and bottom key drivers and the corresponding values.
        Rule 14: For list questions, the narrative must include only the length and dimension data information including the total measure.
        Rule 15: For top-bottom question types, with three or more dimension limit, mention only the top/bottom two values for each dimension in the narrative. Include only the number of many tops and its value if many tops are present for dimension1. Very Important to note that Level2 values in json represents the dimension2 top/bottom values in level1 top1(dimension1).
        Rule 16: Strictly exclude attaching symbols such as "EUR", "USD", "$", "%" etc to any values. The narrative strictly must contain only the information in the Input_question's json. The narrative strictly must not contain any calculated values if any values mentioned in rule to include in narrative is not known.
        Generate the narrative for the given input, ensuring it adheres to these guidelines."""

question = "trend of sales across market in 2021"
intermediate_result = "{'Date_Interval': 'month', 'Latest_period': '2021-12-01', 'Total_Latest_Value': 200609.99, 'Latest_Period_Constant_Measures': False, 'Latest_Top1': ['APAC'], 'Latest_Top1_Values': [53514.51], 'Latest_Top2': ['US'], 'Latest_Top2_Values': [47591.55], 'Latest_Bottom1': ['Canada'], 'Latest_Bottom1_Values': [270.6], 'Negative_present': False, 'Measure': [{'label': 'Sales', 'variable': 'Sales', 'type': 'measure', 'alternate_usage': ['Sales'], 'isRegional': False, 'number_type': 'add_values'}], 'Dimension': [{'label': 'Order Date', 'variable': 'Order Date', 'type': 'dimension', 'alternate_usage': ['Order Date'], 'isRegional': False, 'singular': 'Order Date', 'plural': 'Order Dates'}, {'label': 'Market', 'variable': 'Market', 'type': 'dimension', 'alternate_usage': ['Market'], 'isRegional': False, 'singular': 'Market', 'plural': 'Markets'}], 'Dimension_1_len': 12, 'Dimension_2_len': 7, 'Filter': {'Order Date': [{'ref_interval': 'month', 'start_date': '2021/01/01', 'end_date': '2021/12/31'}]}, 'Many_Top_1': False, 'Share_of_Top1_Latest_Period': '26.68', 'ShareType_of_Top1_Latest_Period': '%', 'Share_of_Top2_Latest_Period': '23.72', 'ShareType_of_Top2_Latest_Period': '%', 'Label_Dates': ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01'], 'Number_of_Tops_Latest_period': 1, 'Additive_Last_Two_Periods_Performance': {'focus_call_out_absolute': False, 'competitor_call_out_absolute': False, 'competitor_available': True, 'focus_competitor_similar_value_change': False, 'competitor_brand': 'US', 'focus_brand': 'APAC', 'focus_same_value_in_final_periods': False, 'competitor_same_value_in_final_periods': False, 'focus_directionv1': 'up', 'focus_change': '37.06', 'focus_change_type': '%', 'competitor_directionv1': 'up', 'com_change': '62.62', 'competitor_change_type': '%'}, 'Top_Players': ['APAC']}"
input_to_model = "Input_question: "+question +"\n"+ str(intermediate_result)
prompt = "Analyse the given input JSON and infer the values as per the need expressed in the Question in input.\n" + str(input_to_model) + "\nNarrative:"
    
message_text = [
    {"role":"system","content":narrative_generation_rules},
    {"role": "user", "content": prompt}
]

In [3]:
response = client.chat.completions.create(
    model="lumin-rnd-gpt-4-32k", # model = "deployment_name".
    messages=message_text,
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=True,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=2,
)

In [4]:
print(response.choices[0].message.content)

In 2021, total sales reached 200,609.99 with APAC and US markets leading at 26.68% and 23.72% shares respectively. The latest sales value for December 2021 was 53514.51 for APAC and 47591.55 for US. Canada had the lowest sales at 270.6.


In [ ]:
import numpy as np
final_prob = 0.0
token_count = 0
for logprob in response.choices[0].logprobs.content:
    token_count+=1
    final_prob+=
    print(f"Token or the word: {logprob.token} and the probability: {logprob.logprob} linear probability: {np.round(np.exp(logprob.logprob)*100,2)}%")